In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
import yaml
import os
from dotenv import load_dotenv
load_dotenv()


class Answer(BaseModel):
    title_en: str = Field(description="")
    title_ko: str = Field(description="")
    contents: str = Field(default="")
    detail_contents: str = Field(description="")
    urls: str = Field(description="")

class QandAChain:
    def __init__(self):
        base_path = os.getcwd() 
        # base_path = os.path.dirna/e(os.path.abspath(__file__))  # 현재 파일의 경로
        file_path = os.path.join(base_path, 'prompt', 'QandAPrompt.yaml')

        #with open(file_path, 'r') as f:
        #    self.__prompt_template = yaml.load(f, Loader=yaml.SafeLoader)['template']
        
        self.__prompt_template = """
        너는 여행 정보를 제공하는 AI야.

        사용자의 질문에 아래 JSON 형식으로 답변해 줘:

        {format_instructions}

        질문: {question}
        """

        self.__model = ChatOpenAI(
            temperature = 0.1,
            model_name="gpt-4o-mini",
            max_tokens = 2048,
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )
        self.__parser = JsonOutputParser(pydantic_object=Answer)
        # self.__parser = StrOutputParser()

    def main_chain_invoke(self, question):
        prompt = PromptTemplate.from_template(self.__prompt_template)
        prompt = prompt.partial(format_instructions=self.__parser.get_format_instructions())

        chain = prompt | self.__model | self.__parser

        result = chain.invoke({"question": question})
        return result

In [8]:
if __name__ == "__main__":
    chain = QandAChain()
    result = chain.main_chain_invoke("LangChain이란?")
    print(result)

{'title_en': 'What is LangChain?', 'title_ko': 'LangChain이란?', 'contents': 'LangChain은 자연어 처리(NLP)와 관련된 다양한 작업을 수행하기 위한 프레임워크입니다.', 'detail_contents': 'LangChain은 언어 모델을 활용하여 데이터 처리, 대화형 AI, 정보 검색 등 다양한 응용 프로그램을 구축할 수 있도록 돕는 도구입니다. 이 프레임워크는 여러 가지 모듈과 구성 요소를 제공하여 개발자가 쉽게 복잡한 NLP 작업을 수행할 수 있게 합니다.', 'urls': 'https://langchain.readthedocs.io/'}


In [ ]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
service_key = os.getenv("TOURAPI_SERVICE_KEY")

url = "https://apis.data.go.kr/B551011/KorService2/areaBasedList2"

params = {
    "MobileOS" : "WEB",
    "MobileApp" : "TOUR_AI",
    "serviceKey": service_key,
    "lDongRegnCd" : "11",
    "lDongSignguCd" : "110",
    "_type": "json"
    "contentTypeId" : "12"
}



res = requests.get(url, params=params)
print(res.status_code)
print(res.text)
res
# print(res.body)/


200
{"response": {"header":{"resultCode":"0000","resultMsg":"OK"},"body": {"items": {"item":[{"addr1":"서울특별시 종로구 평창30길 28","addr2":"(평창동)","areacode":"1","cat1":"A02","cat2":"A0206","cat3":"A02060500","contentid":"129854","contenttypeid":"14","createdtime":"20071106103314","firstimage":"http://tong.visitkorea.or.kr/cms/resource/11/3506711_image2_1.jpg","firstimage2":"http://tong.visitkorea.or.kr/cms/resource/11/3506711_image3_1.jpg","cpyrhtDivCd":"Type1","mapx":"126.9751811398","mapy":"37.6122099878","mlevel":"6","modifiedtime":"20250718090000","sigungucode":"23","tel":"","title":"가나아트센터","zipcode":"03004","lDongRegnCd":"11","lDongSignguCd":"110","lclsSystm1":"VE","lclsSystm2":"VE07","lclsSystm3":"VE070600"},{"addr1":"서울특별시 종로구 인사동10길 11 (관훈동)","addr2":"","areacode":"1","cat1":"A02","cat2":"A0206","cat3":"A02060500","contentid":"129898","contenttypeid":"14","createdtime":"20071106103446","firstimage":"http://tong.visitkorea.or.kr/cms/resource/36/3505536_image2_1.jpg","firstimage2":"htt

<Response [200]>

In [19]:
data = res.json()
items = data["response"]["body"]["items"]["item"]
for item in items:
    print(f"제목: {item.get('title')}")
    print(f"주소: {item.get('addr1')} {item.get('addr2')}")
    print()

제목: 가나아트센터
주소: 서울특별시 종로구 평창30길 28 (평창동)

제목: 가람화랑
주소: 서울특별시 종로구 인사동10길 11 (관훈동) 

제목: 가톨릭대학교 성신교정도서관
주소: 서울특별시 종로구 창경궁로 296-12 (혜화동)

제목: 가회동성당
주소: 서울특별시 종로구 북촌로 57 (가회동) 

제목: 가회민화박물관
주소: 서울특별시 종로구 북촌로 52 (가회동) 

제목: 감꽃당
주소: 서울특별시 종로구 돈화문로11다길 38 

제목: 감로암(서울)
주소: 서울특별시 종로구 율곡로23길 16 (충신동) 

제목: 감미옥
주소: 서울특별시 종로구 종로 166-1 

제목: 강촌쌈밥
주소: 서울특별시 종로구 평창30길 9 

제목: 갗
주소: 서울특별시 종로구 난계로27길 30-8 (숭인동) 1층

